# Homework (10 points)

In this homework we train Sound Event Detection model.

Dataset: https://disk.yandex.ru/d/NRpDIp4jg2ODqg

Hint to download from yadisk. To get real download link for wget or curl you need:
- Open developer console and go to the network tab
- Press download button
- find "download-url" on the network tab and copy

In [ ]:
# download dataset
!wget -O data.tar.gz "https://downloader.disk.yandex.ru/disk/4c5d0c8924b383512ea1af6e7ebe0835dfaf863ca5e1920d7d78076ffedf38f4/65baef2b/gtj3WQiuHGabqHv6W0pVHNHKJOSrL1Ndqw5DdKyJakWvM0_u-aVydaSJ9SFBTCXqkDf5pEJT-jL3E5RuN55B9A%3D%3D?uid=0&filename=sound_event_detection.tar.gz&disposition=attachment&hash=cFvndBq/xOwr18kbjbvCR1M9L3Wj1pzfVE929UmDTjP2u%2BA0B6mbXpTOy5mLcun8q/J6bpmRyOJonT3VoXnDag%3D%3D&limit=0&content_type=application%2Fx-gzip&owner_uid=163052607&fsize=10308787085&hid=51bb9b7fb344746dbead9fd66c87e539&media_type=compressed&tknv=v2"
!tar -xvf data.tar.gz

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tqdm.notebook as tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as torch_data
import torchaudio

# realization of Dataset for given data
import dataset

from IPython.display import clear_output

%matplotlib inline

In [ ]:
DEVICE = 'cpu' # also you can use "cuda" for gpu and "mps" for apple silicon
DATADIR = 'data'
LOADER_WORKERS = 8

In [ ]:
# FBANK 80 by default, but you can choose something else
transform = torchaudio.transforms.MelSpectrogram(n_mels=80)
trainset = dataset.Dataset('train', 'data', transform)
testset = dataset.Dataset('eval', 'data', transform)

### Eval part (1 point)

Write balanced accuracy:
$$BAcc = \frac{1}{classes}\sum_{c = 1}^{classes} \frac{\sum_i^n I(y_i = p_i = c)}{\sum_i^n I(y_i = c)}$$

Where:
- $y_i$ -- target class for $i$ element
- $p_i$ -- predicted class for $i$ element

In [2]:
# Get list of pairs (target_class, predicted_class)
def balanced_accuracy(items: list[tuple[int, int]]) -> float:
    <YOUR CODE IS HERE>

In [5]:
assert np.isclose(balanced_accuracy([(0, 0), (0, 0), (1, 1)]), 1.0)
assert np.isclose(balanced_accuracy([(0, 1), (1, 0)]), 0.0)
assert np.isclose(balanced_accuracy([(0, 0), (0, 0), (1, 0)]), 0.5)

### Train part (9 points)

Train some model with test accuracy > 0.5

You can train any model you want. The only limitation is that it must be trained from scratch on the data provided in the task. For example you can choose model from:
- DNN
- CNN 1d
- CNN 2d
- Transformer
- RNN
- mixes of given models

Hints:
- No need to train large models for this task. 10 million parameters is more than you need.
- Watch to overfitting, try to add Augmentation, Dropout, BatchNorm, L1/L2-Regulatization or something else.
- Use poolings or strides to reduce time-dimenstion. It is better to reduce the dimension gradually rather than at the end.
- Try different features (mel-spec, log-mel-spec, mfcc)

P.S. Points can be deducted for unclear training charts

PP.S. A partial score will be awarded for a test accuracy < 0.5

PPP.S. Add log to Melspectrogram in torchaudio.transform

In [ ]:
def stage(
    model: nn.Module,
    data: dataset.Dataset,
    loss_fun: nn.Module,
    opt: optim.Optimizer,
    batch_size: int = 256,
    train: bool = True
):
    loader = torch_data.DataLoader(
        data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=LOADER_WORKERS,
        collate_fn=dataset.collate_fn
    )
    if train:
        model.train()
    else:
        model.eval()
    loss_sum, batches = 0.0, 0
    pred_pairs = []
    for X, Y in tqdm.tqdm(loader):
        pred = model.forward(X.to(DEVICE))
        loss = loss_fun(pred, Y.to(DEVICE))
        if train:
            opt.zero_grad()
            loss.backward()
            opt.step()
        loss_sum += loss.item()
        batches += 1
        with torch.no_grad():
            pred_pairs.extend(zip(
                Y.data.numpy().reshape(-1),
                torch.argmax(pred, dim=1).cpu().data.numpy().reshape(-1)
            ))
    return loss_sum / batches, balanced_accuracy(pred_pairs)


def train(
    model: nn.Module,
    opt,
    batch_size: int = 256,
    epochs: int = 10,
):
    train_losses, test_losses, train_accs, test_accs = [], [], [], []
    for epoch in range(epochs):
        train_loss, train_acc = stage(model, trainset, opt, batch_size=batch_size)
        test_loss, test_acc = stage(model, testset, opt, batch_size=batch_size, train=False)
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        test_losses.append(test_loss)
        test_accs.append(test_acc)
        clear_output()
        fig, axis = plt.subplots(1, 2, figsize=(15, 7))
        axis[0].plot(np.arange(1, epoch + 2), train_losses, label='train')
        axis[0].plot(np.arange(1, epoch + 2), test_losses, label='test')
        axis[1].plot(np.arange(1, epoch + 2), train_accs, label='train')
        axis[1].plot(np.arange(1, epoch + 2), test_accs, label='test')
        axis[0].set(xlabel='epoch', ylabel='CE Loss')
        axis[1].set(xlabel='epoch', ylabel='Accuracy')
        fig.legend()
        plt.show()
        print(f'Epoch {epoch + 1}.')
        print(f'Train loss {train_loss}. Train accuracy {train_acc}.')
        print(f'Eval loss {train_loss}. Eval accuracy {train_acc}')

In [ ]:
class Model(nn.Module):
    <YOUR CODE IS HERE>

In [ ]:
model = Model()
opt = optim.Adam(model.parameters())
train(model, opt)